In [14]:
import bagpy
from bagpy import bagreader
import os
import pandas as pd
from dateutil.parser import parse as parsedate
import matplotlib.pyplot as plt
from rosbags.highlevel import AnyReader as RosBagReader
from pathlib import Path
import logging
from dotenv import dotenv_values
from dotenv import load_dotenv
import ast 
from functions import *

%load_ext autoreload
%autoreload 2

config = dotenv_values()
data_path = os.path.normpath(config['DATA_PATH'])
os.environ['DATA_PATH'] = data_path
print(f"DATA_PATH is set to: {os.environ['DATA_PATH']}")

rat_number = 7
date = '240822'

messages, bag_file = determine_trial_info(rat_number, date)

extracted_biases_file_path = os.path.join(os.path.dirname(bag_file), 'extracted_biases.csv')

#check whether the files aleady exist. If they exist read them if not create them.
if os.path.exists(extracted_biases_file_path):
    #read the extracted bias csv file that contains trial information
    extracted_biases = pd.read_csv(extracted_biases_file_path)
    #read the trial type summary csv file
    csv_trial_type_summary = os.path.join(os.path.dirname(bag_file), 'trial_type_summary.csv')
    trial_type_summary = pd.read_csv(csv_trial_type_summary)
    print('The dataframes have been created previously')
else:
    filtered_masseages = filter_start_of_trial_messages(messages)

    final_messages = remove_specific_message(filtered_masseages)

    del final_messages[-22:]

    df = create_dataframe_from_messages(final_messages)

    #seperate the 'Left Cue', 'Right Cue', 'Sound Cue' of each Trial (row) and pu them in three new columns
    df[['Left Cue', 'Right Cue', 'Sound Cue']] = df['Trial'].apply(lambda x: pd.Series(extract_trial_info(x)))

    df['Start Chamber Number'] = df['Start Chamber'].str.extract(r'Chamber (\d+)')

    #change the order of the columns 
    new_df = df[['Number', 'Start Chamber Number', 'Left Cue', 'Right Cue', 'Sound Cue', 'Result', 'Choice']]

    new_df['Trial Type'] = new_df.apply(determine_trial_type, axis=1)

    new_df.to_csv(extracted_biases_file_path, index=False)

    print(f"DataFrame saved to {extracted_biases_file_path}")

    # Group by 'Trial Types' and 'Results' and count occurrences
    grouped_df = new_df.groupby(['Trial Type', 'Result']).size().unstack(fill_value=0)

    # Rename columns for clarity
    grouped_df.columns = ['Error_Count', 'Success_Count']

    # Calculate total repetitions of each trial type
    grouped_df['Total_Repetitions'] = grouped_df.sum(axis=1)

    # Reset index to make 'Trial Types' a column again
    grouped_df = grouped_df.reset_index()

    # Create a DataFrame with all 16 trial types and zeros for counts
    all_trial_types = pd.DataFrame({
        'Trial Type': range(1, 17)
    })

    # Merge the all_trial_types DataFrame with grouped_df
    # Use a left join to ensure all trial types are included
    final_df = pd.merge(all_trial_types, grouped_df, on='Trial Type', how='left')

    # Fill NaN values resulting from the merge with 0
    final_df = final_df.fillna(0)

    # Ensure all counts are integers
    final_df['Error_Count'] = final_df['Error_Count'].astype(int)
    final_df['Success_Count'] = final_df['Success_Count'].astype(int)
    final_df['Total_Repetitions'] = final_df['Total_Repetitions'].astype(int)

    # Save the final DataFrame to CSV
    output_csv_path = os.path.join(os.path.dirname(bag_file), 'trial_type_summary.csv')
    final_df.to_csv(output_csv_path, index=False)

    print(f"Summary saved to {output_csv_path}")


past_three_days_file_path = os.path.join(os.path.dirname(bag_file), 'Past_three_days_biases.csv')
#check whether the analysis csv files of the past three days aleady exists. If it exists read it if not create it.
if os.path.exists(past_three_days_file_path):
    #read the extracted bias csv file that contains trial information
    past_three_days_biases = pd.read_csv(past_three_days_file_path)
    print('The dataframes have been created previously')
else:
    combine_csv_files(rat_number, date)







The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
DATA_PATH is set to: \\10.34.1.59\big_gulp\nc4_rat_data\Maze_Rats
Using path: \\10.34.1.59\big_gulp\nc4_rat_data\Maze_Rats
Using path: \\10.34.1.59\big_gulp\nc4_rat_data\Maze_Rats
Looking in folder: \\10.34.1.59\big_gulp\nc4_rat_data\Maze_Rats\NC40007
Looking in folder: \\10.34.1.59\big_gulp\nc4_rat_data\Maze_Rats\NC40007\240822
Error processing: Current trial number: 0 - list index out of range
Error processing: Current trial number: 1 - list index out of range
Error processing: Current trial number: 2 - list index out of range
Error processing: Current trial number: 3 - list index out of range
Error processing: Current trial number: 4 - list index out of range
Error processing: Current trial number: 5 - list index out of range
Error processing: Current trial number: 6 - list index out of range
Error processing: Current trial number: 7 - list index out of range
Error processing: Current trial numbe

C:\Users\afsoo\AppData\Local\Temp\ipykernel_96824\1860168197.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['Trial Type'] = new_df.apply(determine_trial_type, axis=1)


Combined CSV saved to \\10.34.1.59\big_gulp\nc4_rat_data\Maze_Rats\NC40007\240822\Past_three_days_biases.csv
